In [94]:
import numpy as np
import matplotlib.pyplot as plt

In [95]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')
data_general_model = np.load('/content/drive/My Drive/Github/mtc-device-activation/e4_general_model.npz', allow_pickle=True)
data_communication_simulation = np.load('/content/drive/My Drive/Github/mtc-device-activation/e4_communication_simulation.npz', allow_pickle=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
# Access the saved data from the .npz file
all_EventCount = data_general_model['all_EventCount']
all_DeviceActiveCount = data_general_model['all_DeviceActiveCount']
all_DeviceProbability = data_general_model['all_DeviceProbability']
all_DeviceAlarmStatus = data_general_model['all_DeviceAlarmStatus']
all_DeviceStatus = data_general_model['all_DeviceStatus']
all_DeviceLocations = data_general_model['all_DeviceLocations']
all_DeviceLocationsPolar = data_general_model['all_DeviceLocationsPolar']
all_EventLocations = data_general_model['all_EventLocations']
all_EventLocationsPolar = data_general_model['all_EventLocationsPolar']
device_count = data_general_model['device_count']
event_count = data_general_model['event_count']
bs_location = data_general_model['bs_location']
bs_radius = data_general_model['bs_radius']
k = data_general_model['k']
sparsity = data_general_model['sparsity']
num_samples = data_general_model['num_samples']

all_theta = data_communication_simulation['all_theta']
all_gamma = data_communication_simulation['all_gamma']
all_Z = data_communication_simulation['all_Z']
all_noise = data_communication_simulation['all_noise']
all_received_signal = data_communication_simulation['all_received_signal']
seq_length = data_communication_simulation['seq_length']
snr_db_array = data_communication_simulation['snr_db_array']
M = data_communication_simulation['M']

In [97]:
num_samples=1

In [98]:
print(M)

4


SBL

In [102]:
import numpy as np

def sbl_mmv(A, y, N, sig2e, Tau_p, max_iter_sbl=500, stopping_criterion=1e-4):
    """Sparse Bayesian Learning (SBL) for Multiple Measurement Vectors (MMV)."""

    M = y.shape[1]  # Number of antennas

    # Initialize Gamma (covariance matrix of the signal)
    Gamma = np.eye(N) * 0.1  # Initial guess
    xhat_sbl = np.zeros((N, M), dtype=np.complex128)  # Final estimate

    # Iteration over the maximum number of iterations
    for t in range(max_iter_sbl):
        # Compute Sigma_y
        Sigma_y = A @ Gamma @ A.T + sig2e * np.eye(Tau_p)

        # Use Cholesky decomposition for numerical stability in inversion
        Sigma_y_inv = np.linalg.inv(Sigma_y)

        # Compute mean and covariance of x
        mu_x = Gamma @ A.T @ Sigma_y_inv @ y
        Sigma_x = Gamma - Gamma @ A.T @ Sigma_y_inv @ A @ Gamma

        # Update gamma values (vectorized for efficiency)
        gamma_new = (np.linalg.norm(mu_x, axis=1)**2) / M + np.diag(Sigma_x)

        # Check for convergence
        if np.linalg.norm(gamma_new - np.diag(Gamma)) < stopping_criterion:
            print(f"Converged after {t+1} iterations")
            break

        # Update Gamma
        Gamma = np.diag(gamma_new)

    return mu_x, np.diag(Gamma)  # Final estimates


In [100]:
# Placeholder arrays for SBL results
sbl = np.zeros((num_samples, len(snr_db_array), device_count, M), dtype=np.complex128)  # SBL results
gamma_sbl = np.zeros((num_samples, len(snr_db_array), device_count), dtype=np.complex128)  # SBL gamma values
hit_rate_sbl = np.zeros((num_samples, len(snr_db_array)))  # Hit rate for SBL
miss_detection_rate_sbl = np.zeros((num_samples, len(snr_db_array)))  # Miss detection rate for SBL
false_alarm_rate_sbl = np.zeros((num_samples, len(snr_db_array)))  # False alarm rate for SBL
norm_mse_sbl = np.zeros((num_samples, len(snr_db_array)))  # Norm MSE for SBL

In [101]:
from tqdm import tqdm

# Loop through each sample with an outer progress bar
for sample_index in tqdm(range(num_samples), desc="Processing Samples", position=0):
    theta = all_theta[sample_index]
    gamma = all_gamma[sample_index]  # True gamma values (ground truth)

    # Loop over each SNR level with an inner progress bar
    for snr_index in tqdm(range(len(snr_db_array)), desc="Processing SNR Levels", leave=False, position=1):
        received_signal = all_received_signal[sample_index, snr_index]
        Z = all_Z[sample_index]
        snr_db = snr_db_array[snr_index]
        snr = 10 ** (snr_db / 10)
        signal_power = np.mean(np.abs(np.dot(theta, Z))**2)
        noise_power = signal_power / snr

        # Apply SBL algorithm for the current sample and SNR level
        sbl_result, gamma_result = sbl_mmv(theta, received_signal, device_count, noise_power, seq_length)

        # Store the results
        sbl[sample_index, snr_index, :, :] = sbl_result
        gamma_sbl[sample_index, snr_index, :] = gamma_result

        # Avoid division by zero in hit/miss/false alarm rate calculations
        active_devices = np.sum(gamma == 1)  # True active devices
        inactive_devices = np.sum(gamma == 0)  # True inactive devices

        # Calculate the hit rate for SBL
        hit_rate_sbl[sample_index, snr_index] = (
            100 * np.sum((gamma_result == 1) & (gamma == 1)) / active_devices if active_devices > 0 else 0
        )

        # Calculate the miss detection rate for SBL
        miss_detection_rate_sbl[sample_index, snr_index] = (
            100 * np.sum((gamma_result == 0) & (gamma == 1)) / active_devices if active_devices > 0 else 0
        )

        # Calculate the false alarm rate for SBL
        false_alarm_rate_sbl[sample_index, snr_index] = (
            100 * np.sum((gamma_result == 1) & (gamma == 0)) / inactive_devices if inactive_devices > 0 else 0
        )

        # Calculate the normalized MSE for SBL
        norm_mse_sbl[sample_index, snr_index] = (
            np.linalg.norm(Z - sbl_result)**2 / max(1e-10, np.linalg.norm(Z)**2)
        )  # Avoid division by zero


Processing SNR Levels:   0%|          | 0/6 [00:00<?, ?it/s]

400



Processing Samples:   0%|          | 0/1 [00:03<?, ?it/s]


ValueError: could not broadcast input array from shape (400,4) into shape (400,400)

In [ ]:
# Compute the average across all samples for each SNR level
avg_hit_rate_sbl = np.mean(hit_rate_sbl, axis=0)  # Shape: (len(snr_db_array),)
avg_miss_detection_rate_sbl = np.mean(miss_detection_rate_sbl, axis=0)  # Shape: (len(snr_db_array),)
avg_false_alarm_rate_sbl = np.mean(false_alarm_rate_sbl, axis=0)  # Shape: (len(snr_db_array),)
avg_norm_mse_sbl = np.mean(norm_mse_sbl, axis=0)  # Shape: (len(snr_db_array),)

# Print or log the results
print("Average Hit Rate across samples per SNR level:", avg_hit_rate_sbl)
print("Average Miss Detection Rate across samples per SNR level:", avg_miss_detection_rate_sbl)
print("Average False Alarm Rate across samples per SNR level:", avg_false_alarm_rate_sbl)
print("Average Normalized MSE across samples per SNR level:", avg_norm_mse_sbl)

In [ ]:
import matplotlib.pyplot as plt

# Define the SNR values for x-axis
snr_values = snr_db_array

# Create subplots for different performance metrics
plt.figure(figsize=(12, 8))

# Plot Hit Rate
plt.subplot(2, 2, 1)
plt.plot(snr_values, avg_hit_rate_sbl, marker='o', linestyle='-', color='b', label='Hit Rate')
plt.xlabel("SNR (dB)")
plt.ylabel("Hit Rate (%)")
plt.title("Hit Rate vs SNR")
plt.grid(True)
plt.legend()

# Plot Miss Detection Rate
plt.subplot(2, 2, 2)
plt.plot(snr_values, avg_miss_detection_rate_sbl, marker='s', linestyle='-', color='r', label='Miss Detection Rate')
plt.xlabel("SNR (dB)")
plt.ylabel("Miss Detection Rate (%)")
plt.title("Miss Detection Rate vs SNR")
plt.grid(True)
plt.legend()

# Plot False Alarm Rate
plt.subplot(2, 2, 3)
plt.plot(snr_values, avg_false_alarm_rate_sbl, marker='^', linestyle='-', color='g', label='False Alarm Rate')
plt.xlabel("SNR (dB)")
plt.ylabel("False Alarm Rate (%)")
plt.title("False Alarm Rate vs SNR")
plt.grid(True)
plt.legend()

# Plot Normalized MSE
plt.subplot(2, 2, 4)
plt.plot(snr_values, avg_norm_mse_sbl, marker='d', linestyle='-', color='m', label='Normalized MSE')
plt.xlabel("SNR (dB)")
plt.ylabel("Normalized MSE")
plt.title("Normalized MSE vs SNR")
plt.yscale('log')  # Log scale for better visualization
plt.grid(True)
plt.legend()

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_sbl_vs_ground_truth(sbl_results, Z, sample_index, snr_index, snr_value):
    """Plot SBL reconstructed signal vs ground truth (Z) for a single sample at a given SNR."""
    plt.figure(figsize=(10, 5))

    # Ground truth (Z)
    plt.plot(np.abs(Z[sample_index,:,1]), marker='s', linestyle='--', color='g', label="Ground Truth (Z)")

    # SBL Estimated Signal
    plt.plot(np.abs(sbl_results[sample_index, snr_index, :, 0]), marker='o', linestyle='-', color='b', label="SBL Estimate")

    plt.xlabel("Device Index")
    plt.ylabel("Magnitude of Signal")
    plt.title(f"SBL vs. Ground Truth (Sample {sample_index}, SNR {snr_value} dB)")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
sample_index = 0  # Choose a specific sample
snr_index = 2  # Choose a specific SNR index
snr_value = snr_db_array[snr_index]

plot_sbl_vs_ground_truth(sbl, all_Z, sample_index, snr_index, snr_value)